Action1	"使用MinHashLSHForest对微博新闻句子进行检索 weibo.txt
针对某句话进行Query，查找Top-3相似的句子"

In [1]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [2]:
# 读取文件
f = open('weibos.txt', 'r', encoding='UTF-8')
text = f.read()

In [3]:
# 以句号，叹号，问号作为分隔，去掉\n换行符号
sentences = re.split('[#。!？]', text.replace('\n', ''))

In [4]:
if sentences[len(sentences)-1] == '':
    sentences.pop()

In [5]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str

In [6]:
# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [7]:
# 设置停用词
stop = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]

In [8]:
# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpg1hmbafb' -> '/tmp/jieba.cache'
Loading model cost 0.925 seconds.
Prefix dict has been built successfully.


In [9]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)

In [10]:
# index所有key，以便可以进行检索
forest.index()

In [11]:
query = '国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职'

In [12]:
# 将item_text进行分词
item_str = get_item_str(query)

In [13]:
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

In [14]:
# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 3)

In [15]:
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))

24 0.4765625 国足输给叙利亚后，里皮坐不住了，直接辞职了
2 0.578125 国足输给叙利亚之后，里皮辞职
28 1.0 国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职


In [16]:
print("Top 3 邻居", result)

Top 3 邻居 [24, 2, 28]
